In [ ]:
from PIL import Image
import numpy as np
import torch
from fastai.vision.models import Darknet


from gradcam.misc_functions import save_class_activation_images
from gradcam.gradcam import GradCam


In [ ]:

# Get params
target_example = 1  # Lato


In [ ]:
def preprocess_image(pil_im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): Image to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # Resize image
    if resize_im:
        pil_im.thumbnail((512, 512))
    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = torch.autograd.Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [ ]:
# (original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
#     get_example_params(target_example)

example_list = (('/home/paperspace/code/fontastic/data/stratified/train/Lora/Lora-Regular_300_rand_crop_8.jpg', 2),
                    ('/home/paperspace/code/fontastic/data/stratified/train/Lato/Lato-Regular_300_rand_crop_6.jpg', 1),
                ('/home/paperspace/code/fontastic/data/stratified/train/Fahkwang/Fahkwang-BoldItalic_150_rand_crop_3.jpg', 1),
                ('/home/paperspace/code/fontastic/data/stratified/train/Merriweather/Merriweather-Black_50_rand_crop_7.jpg', 1)
                )
example_index = 3

img_path = example_list[example_index][0]
target_class = example_list[example_index][1]
file_name_to_export = img_path[img_path.rfind('/')+1:img_path.rfind('.')]
# Read image
original_image = Image.open(img_path).convert('RGB')
# Process image
prep_img = preprocess_image(original_image)
# Define model



In [ ]:
import torch.nn as nn

In [ ]:
# m = Darknet([1, 2, 4, 6, 3], num_classes=6, nf=16)
# m = nn.DataParallel(m, device_ids=None)

# m.load_state_dict(torch.load('/home/paperspace/code/fontastic/notebooks/models/darknet-params'))


In [ ]:
from torchvision import models
import os

In [ ]:
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

In [ ]:
# model_ft = models.resnet50(pretrained=True)

# num_frts = model_ft.fc.in_features
# model_ft.avgpool.output_size = 1
# # # model_ft.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,num_frts))
# model_ft.fc = nn.Linear(num_frts, 6)

model_ft = models.resnet50(pretrained=True)
num_frts = model_ft.fc.in_features
model_ft.fc = nn.Linear(in_features=num_frts, out_features=6)


# m = nn.DataParallel(model_ft, device_ids=None)

# m.load_state_dict(torch.load('/home/paperspace/code/fontastic/notebooks/models/darknet-params'))


In [ ]:
experiments_path = '/home/paperspace/code/fontastic/data/stratified/'

In [ ]:
zzz = torch.load('/home/paperspace/code/fontastic/data/stratified/model_resnet50_tfms.pth')

In [ ]:
model_ft.load_state_dict(zzz.state_dict())

In [ ]:
m = model_ft

In [ ]:
m = m.to(torch.device("cpu"))

In [ ]:
# Grad cam
grad_cam = GradCam(m, target_layer=4)


In [ ]:
device = torch.device("cpu")

prep_img.to(device).shape

In [ ]:
# Generate cam mask
cam = grad_cam.generate_cam(prep_img, target_class)



In [ ]:
f'{file_name_to_export}-gradcam-resnet'

In [ ]:
curr_img = f'{file_name_to_export}-gradcam-resnet'

In [ ]:
im = Image.fromarray(cam)

In [ ]:
save_class_activation_images(original_image, cam, f'{file_name_to_export}-gradcam-resnet')
print('Grad cam completed')

In [ ]:
Image.open('../results/Merriweather-Black_50_rand_crop_7-gradcam-resnet_Cam_On_Image.png')